<img src="img/big_day_proj_head.png" align="right">

<h1><center>AVACADO PRICING</center></h1>
<h1><center>Time Series Cleaning</center></h1>

Analysis by [**Frank Flavell**](https://www.linkedin.com/in/m-frank-flavell/)

## Data Overview

This dataset was provided by Justin Higgins on [Kaggle.com](https://www.kaggle.com/neuromusic/avocado-prices/data) and he sourced it from the [Hass Avacado Board](https://hassavocadoboard.com/).  Below is a descrition of the data from Justin.

<blockquote>The dataset contains weekly 2018 retail scan data for national retail volume (units) and price. Retail scan data comes directly from retailers’ cash registers based on actual retail sales of Hass avocados. Starting in 2015, the table below reflects an expanded, multi-outlet retail data set. Multi-outlet reporting includes an aggregation of the following channels: grocery, mass, club, drug, dollar and military. The Average Price (of avocados) in the table reflects a per unit (per avocado) cost, even when multiple units (avocados) are sold in bags. The Product Lookup codes (PLU’s) in the table are only for Hass avocados. Other varieties of avocados (e.g. greenskins) are not included in this table.</blockquote>

### Features
* Date - The date of the observation
* AveragePrice - the average price of a single avocado
* type - conventional or organic
* year - the year
* Region - the city or region of the observation
* Total Volume - Total number of avocados sold
* 4046 - Total number of avocados with PLU 4046 sold
* 4225 - Total number of avocados with PLU 4225 sold
* 4770 - Total number of avocados with PLU 4770 sold
* total_bags - total number of bags sold
* small_bags - total number of small bags sold
* large_bags - total number of large bags sold
* xlarge_bags - total number of extra large bags sold


## Cleaning Process<span id="0"></span>

1. [**Data Import & Header Formatting**](#1)
<br/><br/>
2. [**Check for Missing Values**](#2)
    * Check null values
    * Check regions
    * Identify missing rows
    * Remove unnecessary data
    * Examine duplicate date ranges for each region
<br/><br/>
3. [**Convert Datatypes**](#3)
    * Convert string dates to DateTime object
    * Move date to index
<br/><br/>
4. [**Remove Unnecessary Columns**](#4)
<br/><br/>
5. [**Pickle the DataFrame**](#5)
<br/><br/>
6. [**Additional Resources**](#6)

## Results

The cleaned dataset includes the average retail price of both conventional and organic avacados for each week between Dec. 27, 2015 and Jan. 7, 2018 for 53 regions across the continental U.S.A.

## Video Tutorial

Coming soon...

# Package Import

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

## <span id="1"></span>1. Data Import
#### [Return Contents](#0)

In [160]:
df = pd.read_csv("avocado.csv", index_col=0)

In [161]:
df.head(2)

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany


## Header Formatting

Making the column names lower case and removing spaces will make it easier to work with the data and allow us to use dot notation when indexing the dataframe.

In [162]:
# Make column names lower case and replace spaces with _
df = df.rename(columns=lambda x: x.lower().replace(' ', '_'))

In [163]:
df.head(2)

,date,averageprice,total_volume,4046,4225,4770,total_bags,small_bags,large_bags,xlarge_bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany


Also renaming long column names and changing confusing names will help with coding efficiency and clarity.

In [164]:
df = df.rename(columns={'averageprice' : 'avg_price'})

In [165]:
df.head(2)

,date,avg_price,total_volume,4046,4225,4770,total_bags,small_bags,large_bags,xlarge_bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany


## <span id="2"></span>2. Check for Missing Values
#### [Return Contents](#0)

In [166]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18249 entries, 0 to 11
Data columns (total 13 columns):
date            18249 non-null object
avg_price       18249 non-null float64
total_volume    18249 non-null float64
4046            18249 non-null float64
4225            18249 non-null float64
4770            18249 non-null float64
total_bags      18249 non-null float64
small_bags      18249 non-null float64
large_bags      18249 non-null float64
xlarge_bags     18249 non-null float64
type            18249 non-null object
year            18249 non-null int64
region          18249 non-null object
dtypes: float64(9), int64(1), object(3)
memory usage: 1.9+ MB


There aren't any null objects in the dataset.  However, it looks like there's something funny with the index values.  Let's take a closer look.

In [167]:
df['date'][0]

0    2015-12-27
0    2015-12-27
0    2015-12-27
0    2015-12-27
0    2015-12-27
        ...    
0    2018-03-25
0    2018-03-25
0    2018-03-25
0    2018-03-25
0    2018-03-25
Name: date, Length: 432, dtype: object

There appears to be 432 "0" indices ranging in date from december 2015 to March 2018.  This is because the 18,249 observations in the dataframe are divided between different regions.  Let's take a look at the different regions.

In [168]:
regions = df.groupby('region').count()

In [169]:
regions

,date,avg_price,total_volume,4046,4225,4770,total_bags,small_bags,large_bags,xlarge_bags,type,year
region,,,,,,,,,,,,
Albany,338,338,338,338,338,338,338,338,338,338,338,338
Atlanta,338,338,338,338,338,338,338,338,338,338,338,338
BaltimoreWashington,338,338,338,338,338,338,338,338,338,338,338,338
Boise,338,338,338,338,338,338,338,338,338,338,338,338
Boston,338,338,338,338,338,338,338,338,338,338,338,338
BuffaloRochester,338,338,338,338,338,338,338,338,338,338,338,338
California,338,338,338,338,338,338,338,338,338,338,338,338
Charlotte,338,338,338,338,338,338,338,338,338,338,338,338
Chicago,338,338,338,338,338,338,338,338,338,338,338,338


In [170]:
regions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 54 entries, Albany to WestTexNewMexico
Data columns (total 12 columns):
date            54 non-null int64
avg_price       54 non-null int64
total_volume    54 non-null int64
4046            54 non-null int64
4225            54 non-null int64
4770            54 non-null int64
total_bags      54 non-null int64
small_bags      54 non-null int64
large_bags      54 non-null int64
xlarge_bags     54 non-null int64
type            54 non-null int64
year            54 non-null int64
dtypes: int64(12)
memory usage: 5.5+ KB


There are 54 different regions, each with 338 observations, which adds up to 18,252.  However, when we check this against the number of observations in our main DataFrame, which has 18,249, it appears we are missing 3 observations.  Take a closer look at the groupby object and you'll see that WestTexNewMexico is missing 3 observations compared to the rest.  They didn't show up as null values, because these entire rows weren't even in the dataset.

Let's figure out which 3 observations are missing.

We start by making a list of the dates in WestTexNewMexico.

In [171]:
wtnm = df[df.region == "WestTexNewMexico"]

In [172]:
wtnm_dates = list(wtnm.date)

In [173]:
len(wtnm_dates)

335

In [174]:
len(set(wtnm_dates))

169

Interestingly, when we make a list of the dates in the WestTexNewMexico region, it appears that there are duplicate dates after turning the list into a set.

Next we will make a list of another region that has the full 338 observations.

In [175]:
west = df[df.region == "West"]

In [176]:
west_dates = list(west.date)

In [177]:
len(west_dates)

338

In [178]:
len(set(west_dates))

169

There are also duplicates in this list.  And when I compare the number of unique dates in the West list with the number of unique dates in the WestTexNewMexico list, they have the same length.  This means that they both have the same dates, but they a missing a few duplicate.

I need to determine the difference between the two lists using a method that is sensitive to duplicates.

In [179]:
from collections import Counter
res = list((Counter(west_dates) - Counter(wtnm_dates)).elements())
res

['2015-12-06', '2017-06-25', '2017-06-18']

Now that I've determined the missing values, we need to decide what to do.  Unfortunately the dates aren't consequitive so we can't drop the same three rows across all regions.  Time Series needs the same interval between each observation for the analysis and modeling to be accurate.

#### Back to the Source

We could go back to Hass Avacado Board and see if these three rows are available, but considering this data from the HAB, I don't think we can recover them.

#### .fillna( )

We can also use the **.fillna( )** method along with methods like **.bfill( )** of **.ffill( )** to fill in missing values. **.bfill( )** (backward filling) looks for the next valid entry in the time series and fills the gaps with this value. Similarly, **.ffill( )** can be used to copy forward the previous valid entry of the time series.

However, it would be a stretch to fill 3 entire rows, especially when I only have 54 observations per region.

For my purposes, I'm going to drop the region so it doesn't affect the rest of my analysis.

In [180]:
df = df[df.region != "WestTexNewMexico"]

In [181]:
df.region.unique()

array(['Albany', 'Atlanta', 'BaltimoreWashington', 'Boise', 'Boston',
       'BuffaloRochester', 'California', 'Charlotte', 'Chicago',
       'CincinnatiDayton', 'Columbus', 'DallasFtWorth', 'Denver',
       'Detroit', 'GrandRapids', 'GreatLakes', 'HarrisburgScranton',
       'HartfordSpringfield', 'Houston', 'Indianapolis', 'Jacksonville',
       'LasVegas', 'LosAngeles', 'Louisville', 'MiamiFtLauderdale',
       'Midsouth', 'Nashville', 'NewOrleansMobile', 'NewYork',
       'Northeast', 'NorthernNewEngland', 'Orlando', 'Philadelphia',
       'PhoenixTucson', 'Pittsburgh', 'Plains', 'Portland',
       'RaleighGreensboro', 'RichmondNorfolk', 'Roanoke', 'Sacramento',
       'SanDiego', 'SanFrancisco', 'Seattle', 'SouthCarolina',
       'SouthCentral', 'Southeast', 'Spokane', 'StLouis', 'Syracuse',
       'Tampa', 'TotalUS', 'West'], dtype=object)

Fare thee well, WestTexasNewMexico.

## Examine the Duplicate Dates in Each Region

Now that I've delt with the missing rows, I'm curious to see why there are duplicate rows.  I know that I'll need to include the region column so that I can compare the time series data between the date ranges for each region.  But what other variable requires there be two versions of the same date range for each region?

I'll start by looking at one region.

In [182]:
albany = df[df.region == 'Albany']

Now, I'll choose a date within the range and compare it with its duplicate to determine the difference between the two.

In [187]:
albany[albany.date == '2018-02-18']

,date,avg_price,total_volume,4046,4225,4770,total_bags,small_bags,large_bags,xlarge_bags,type,year,region
5,2018-02-18,1.43,85630.24,5499.73,61661.76,75.0,18393.75,15677.67,2336.08,380.0,conventional,2018,Albany
5,2018-02-18,1.43,7566.17,4314.30,251.85,0.0,3000.02,3000.02,0.00,0.0,organic,2018,Albany


It appears this is where the convential and organic classification fits in.  There are two sets of time series data for avacados grown using conventional methods as well as organic methods for the same date range and region.  We will need to include this variable so we can further subset the data.

## <span id="3"></span>3. Convert Datatypes
#### [Return Contents](#0)

There are 3 columns with the object datatype instead of int or float, including the date column.  Let's take a closer look.

Since this project is about predicting the price of avacados, the only column with an object datatype we need to convert is the date column.  The avg_price column, which contains the float datatype, is aleady in a form we can work with for time series.

When converting a string make sure Python understands the date correctly, a format argument can be passed as specified in the documentation.

In [188]:
all_dates = list(df['date'])

In [189]:
type(all_dates[0])

str

Because of all the duplicates with the same index values, I turned the column into a list so I could determine that the datatype of the current date is string, even though it is already in the correct DateTime format of a timestamp.  I need to convert the dates into a DateTime object.

You can learn more about the DateTime object from the [Panda's documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html).



In [190]:
# Convert Date to a datetime column
df['date'] = pd.to_datetime(df['date'])

In [191]:
df['date']

0    2015-12-27
1    2015-12-20
2    2015-12-13
3    2015-12-06
4    2015-11-29
        ...    
7    2018-02-04
8    2018-01-28
9    2018-01-21
10   2018-01-14
11   2018-01-07
Name: date, Length: 17914, dtype: datetime64[ns]

As you can see above, the dtype is now DateTime64.

### Switch Dates to Index

Next we need to move the dates into the index and remove unnecessary rows.

In [192]:
# Make Date the index 
df.set_index('date', inplace=True)

In [193]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 17914 entries, 2015-12-27 to 2018-01-07
Data columns (total 12 columns):
avg_price       17914 non-null float64
total_volume    17914 non-null float64
4046            17914 non-null float64
4225            17914 non-null float64
4770            17914 non-null float64
total_bags      17914 non-null float64
small_bags      17914 non-null float64
large_bags      17914 non-null float64
xlarge_bags     17914 non-null float64
type            17914 non-null object
year            17914 non-null int64
region          17914 non-null object
dtypes: float64(9), int64(1), object(2)
memory usage: 1.8+ MB


When I check to make sure the change has occured, we can see the dates are now in the index range.

## <span id="4"></span>4. Remove Unecessary Rows
#### [Return Contents](#0) 

After exploring the data, I've determined that the only rows we need are the dates, now in the index, and the avg_price of avacados as this is the target variable I will attempt to forecast as well as the region and type for subsetting.

In [153]:
df.columns

Index(['avg_price', 'total_volume', '4046', '4225', '4770', 'total_bags',
       'small_bags', 'large_bags', 'xlarge_bags', 'type', 'year', 'region'],
      dtype='object')

In [194]:
df.drop(columns=['total_volume', '4046', '4225', '4770', 'total_bags',
       'small_bags', 'large_bags', 'xlarge_bags', 'year'], inplace=True)

In [195]:
df.head(2)

,avg_price,type,region
date,,,
2015-12-27,1.33,conventional,Albany
2015-12-20,1.35,conventional,Albany


## <span id="5"></span>5. Pickle the Data
#### [Return Contents](#0)

In [196]:
df.to_pickle('avacado_avg_price.pickle')

Check export worked.

In [200]:
pd.read_pickle('avacado_avg_price.pickle')

,avg_price,type,region
date,,,
2015-12-27,1.33,conventional,Albany
2015-12-20,1.35,conventional,Albany
2015-12-13,0.93,conventional,Albany
2015-12-06,1.08,conventional,Albany
2015-11-29,1.28,conventional,Albany
...,...,...,...
2018-02-04,1.41,organic,West
2018-01-28,1.80,organic,West
2018-01-21,1.83,organic,West


## <span id="6"></span>6. Additional Resources
#### [Return Contents](#0)

If you're interested in learning more, check out the next notebook in this series, Time Series EDA, where I will demonstrate ways to visualize time series data and also a 3 step check for stationarity.

You can learn more about Panda's Time Series Functionality by reading the [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html).

You can also learn from [Laura Fedoruk's Medium Post](https://towardsdatascience.com/basic-time-series-manipulation-with-pandas-4432afee64ea), "Basic Time Series Data Manipulation Using Pandas".


**Connect with me at Big Day**

[<img src="img/frank_contact_card_button3.png" align="center">](http://www.bigday.ai)